# Azure

This notebook goes over how to connect to an Azure hosted OpenAI endpoint

In [2]:
from langchain.chat_models import AzureChatOpenAI
from langchain.schema import HumanMessage

In [4]:
BASE_URL = "https://${TODO}.openai.azure.com"
API_KEY = "..."
DEPLOYMENT_NAME = "chat"
model = AzureChatOpenAI(
    openai_api_base=BASE_URL,
    openai_api_version="2023-05-15",
    deployment_name=DEPLOYMENT_NAME,
    openai_api_key=API_KEY,
    openai_api_type="azure",
)

In [5]:
model(
    [
        HumanMessage(
            content="Translate this sentence from English to French. I love programming."
        )
    ]
)

AIMessage(content="\n\nJ'aime programmer.", additional_kwargs={})

## Model Version
Azure OpenAI responses contain `model` property, which is name of the model used to generate the response. However unlike native OpenAI responses, it does not contain the version of the model, which is set on the deployment in Azure. This makes it tricky to know which version of the model was used to generate the response, which as result can lead to e.g. wrong total cost calculation with `OpenAICallbackHandler`.

To solve this problem, you can pass `model_version` parameter to `AzureChatOpenAI` class, which will be added to the model name in the llm output. This way you can easily distinguish between different versions of the model.

In [ ]:
from langchain.callbacks import get_openai_callback

In [14]:
BASE_URL = "https://{endpoint}.openai.azure.com"
API_KEY = "..."
DEPLOYMENT_NAME = "gpt-35-turbo" # in Azure, this deployment has version 0613 - input and output tokens are counted separately

In [15]:
model = AzureChatOpenAI(
    openai_api_base=BASE_URL,
    openai_api_version="2023-05-15",
    deployment_name=DEPLOYMENT_NAME,
    openai_api_key=API_KEY,
    openai_api_type="azure",
)
with get_openai_callback() as cb:
    model(
        [
            HumanMessage(
                content="Translate this sentence from English to French. I love programming."
            )
        ]
    )
    print(f"Total Cost (USD): ${format(cb.total_cost, '.6f')}") # without specifying the model version, flat-rate 0.002 USD per 1k input and output tokens is used


Total Cost (USD): $0.000054


We can provide the model version to `AzureChatOpenAI` constructor. It will get appended to the model name returned by Azure OpenAI and cost will be counted correctly.

In [17]:
model0613 = AzureChatOpenAI(
    openai_api_base=BASE_URL,
    openai_api_version="2023-05-15",
    deployment_name=DEPLOYMENT_NAME,
    openai_api_key=API_KEY,
    openai_api_type="azure",
    model_version="0613"
)
with get_openai_callback() as cb:
    model0613(
        [
            HumanMessage(
                content="Translate this sentence from English to French. I love programming."
            )
        ]
    )
    print(f"Total Cost (USD): ${format(cb.total_cost, '.6f')}")


Total Cost (USD): $0.000044
